**[2019.11.21.] Azure와 함께하는 AutoML || SBA 빅데이터 사이언스 실무연계 프로젝트 || 전미정**

# Azure와 함께하는 Automated Machine Learning_LocalMachine 💻
**손글씨 이미지 분류 모델**

<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png">


* **Part1. 개발 환경 설정**  
* **Part2. 학습 데이터 가져오기**  
* **Part3. 모델 학습**  
* **Part4. 모델 결과**  
* **Part5. 모델 테스트**  

---

## Part1. 개발 환경 설정

1. Workspace 연결
2. Experiment 생성

### 1-1. Workspace 연결

In [ ]:
import azureml
from azureml.core import Workspace
from azureml.core.runconfig import RunConfiguration

# Azure core SDK version 확인
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# 다운 받은 config.json에서 복붙해주세요!

subscription_id = 'subscription_id'
resource_group  = 'resource_group'
workspace_name  = 'workspace_name'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')
    
# 계정 인증문구가 뜨면 하단에 영문과 숫자로 이루어진 코드 9자리를 복사한 후 링크(https://microsoft.com/devicelogin)을 클릭해 붙여넣어주세요.

### 1-2. Experiment 생성

In [ ]:
from azureml.core.experiment import Experiment
import pandas as pd

# 실험명 및 프로젝트 파일 디렉토리 설정
experiment_name = 'automl_digit'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name

pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

---

## Part2. 학습 데이터 가져오기

숫자 이미지 데이터 가져오기

In [ ]:
# 학습 데이터 다운로드
from sklearn import datasets
from matplotlib import pyplot as plt
import numpy as np

digits = datasets.load_digits()

X_train = digits.data[100:,:]
y_train = digits.target[100:]
images = digits.images[100:]

In [ ]:
# 학습 데이터 확인

count = 0
sample_size = 10
plt.figure(figsize = (8, 8))
for index in np.random.choice(len(y_train), sample_size, replace = False):
    count = count + 1
    label = y_train[index]
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=2, y=12, s=label, fontsize=20)
    plt.imshow(images[index], cmap=plt.cm.Greys)

plt.show()

---

## Part3. 모델 학습

`AutoMLConfig` 객체 생성

Experiment를 실행하는데 사용되는 설정값 및 데이터 정의

|Property|Description|
|-|-|
|**task**|classification/regression/forecast|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], 다중 클래스|
|**primary_metric**|<i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**iteration_timeout_minutes**|iteration 1회 소요 시간제한(분)|
|**max_time_sec**|최대 학습 시간(초)(will be deprecated)|
|**iterations**|학습 반복 횟수|
|**preprocess**|데이터 전처리 여부|
|**blacklist_models**|사용하지 않을 모델|
|**whitelist_models**|꼭 사용할 모델|


**primary_metric에 대한 자세한 내용은 [여기](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric/?WT.mc_id=AI-MVP-5003262)를 참고해주세요** 

**Task에 따른 model 종류는 [여기](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#select-your-experiment-type/?WT.mc_id=AI-MVP-5003262)를 참고해주세요**

In [ ]:
from azureml.train.automl import AutoMLConfig
import logging

automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations":10,
    "primary_metric": 'AUC_weighted',
    "preprocess": True,
    "max_concurrent_iterations": 5,
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             X = X_train,
                             y = y_train,
                             **automl_settings,
                            )

In [ ]:
# submit method

local_run = experiment.submit(automl_config, show_output = True)

---

## Part4. 학습 결과

`widgets`은 1분에 한번씩 업데이트됩니다.  
위젯 아래쪽에 세부 정보를 확인할 수 있는 링크가 제공됩니다.

In [ ]:
# Azure Run 위젯 실행

from azureml.widgets import RunDetails
RunDetails(local_run).show()

In [ ]:
# 베스트 모델 확인

best_run, fitted_model = local_run.get_output()
print("Best Model:", best_run)
print("=========")
print("Fitted Model:", fitted_model)

In [ ]:
# metric 종류에 따른 best run model

lookup_metric = "log_loss"
best_run, fitted_model = local_run.get_output(metric = lookup_metric)
print("Best Model:", best_run)
print("=========")
print("Fitted Model:", fitted_model)

---

## Part5. 모델 테스트

In [ ]:
# 테스트 데이터 불러오기

digits = datasets.load_digits()
X_test = digits.data[:10, :]
y_test = digits.target[:10]
images = digits.images[:10]

In [ ]:
predicted = fitted_model.predict(X_test)

In [ ]:
# 랜덤으로 10개 선택 및 테스트
  
count = 0
sample_size = 10
plt.figure(figsize = (8, 8))
for index in np.random.choice(len(y_test), sample_size, replace = False):
    count = count + 1
    label = y_test[index]
    title = "Label value = %d  Predicted value = %d " % (label, predicted[index])
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=2, y=12, s=label, fontsize=20)
    plt.imshow(images[index], cmap=plt.cm.Greys)
plt.show()

잘 되었나요? 수고하셨습니다! 🤗 👏🏻 🥳

---